In [1]:
using Pkg
#Pkg.add("JuMP")
#Pkg.add("HiGHS")
#Pkg.add("Gurobi")
#Pkg.add(url ="https://github.com/rafaelmartinelli/BPPLib.jl")
#Pkg.add("Random")
#Pkg.add("Statistics")


In [2]:
using JuMP
using HiGHS
using BPPLib
using Random
using Statistics

struct Pattern
    x::Vector{Int}
end

function addpattern!(P::Vector{Pattern}, x::Vector{Int})
    if sum(x) == 0; 
        return; 
    end
    for p in P
        if p.x == x; 
            return; 
        end
    end
    push!(P, Pattern(copy(x)))
end

function greedy_fill!(r::Vector{Int}, l::Vector{Int}, L::Int)
    n = length(l)
    x = zeros(Int, n)
    used = 0
    while true
        idx = [j for j in 1:n if r[j] > 0 && used + l[j] <= L]
        isempty(idx) && break
        jbest = argmax([(l[j], r[j]) for j in idx])
        j = idx[jbest]
        x[j] += 1
        r[j] -= 1
        used += l[j]
    end
    return x
end

function build_patterns(l::Vector{Int}, d::Vector{Int}, L::Int; m_max::Int=200)
    n = length(l)
    P = Pattern[]

    for j in 1:n
        if l[j] <= L
            x = zeros(Int, n); x[j] = 1; addpattern!(P, x)
        end
    end
    for a in 1:n, b in a:n
        if l[a] + l[b] <= L
            x = zeros(Int, n); x[a] += 1; x[b] += 1; addpattern!(P, x)
        end
    end
    r = copy(d)
    k = 0
    while k < m_max && sum(r) > 0
        x = greedy_fill!(r, l, L)
        addpattern!(P, x)
        k += 1
    end
    return P
end

function solve_P1(L::Int, l::Vector{Int}, d::Vector{Int}, P::Vector{Pattern};
                  timelimit::Float64=300.0)
    T = length(P)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    set_attribute(model, "time_limit", timelimit)

    @variable(model, y[1:T] >= 0, Int)
    @objective(model, Min, sum(y[t] for t in 1:T))
    @constraint(model, cover[j=1:length(l)], sum(P[t].x[j]*y[t] for t in 1:T) >= d[j])

    optimize!(model)
    obj = objective_value(model)
    status = termination_status(model)
    time_s = JuMP.MOI.get(model, JuMP.MOI.SolveTimeSec())
    return obj, status, time_s, value.(y)
end

function toy_example()
    L = 5
    l = [2, 2, 3]
    d = [4, 3, 1]
    P = build_patterns(l, d, L; m_max=50)
    obj, status, t, _ = solve_P1(L, l, d, P)
    println("  Objective = ", obj, "   Status = ", status, "   Time = ", round(t, digits=3), " s")
end

toy_example()


function write_csp_instance(filename::String, name::String, L::Int, l::Vector{Int}, d::Vector{Int})
    open(filename, "w") do io
        println(io, length(l), " ", L)
        for j in 1:length(l)
            println(io, l[j], " ", d[j])
        end
    end
end

function generate_csp_instance()
    n = 100
    l = rand(2:30, n)
    d = rand(1:10, n)
    L = Int(round(mean(l) * 4))   #keep patterns feasible
     
    return L, l, d,n
end

function solve_csp_file(filename::String; m_max::Int=300)
    data = BPPLib.loadCSP(filename)

    L = Int(data.capacity)
    l = Vector{Int}(data.weights)
    d = Vector{Int}(data.demands)
    println("  Capacity = ", L, " | n = ", length(l))

    P = build_patterns(l, d, L; m_max=m_max)
    obj, status, t, _ = solve_P1(L, l, d, P)

    println("  Objective (primal bound): ", obj)
    println("  Status: ", status, " | Time: ", round(t, digits=2), " s")
end


                
L_big, l_big, d_big,n = generate_csp_instance()
write_csp_instance("my_instance.txt", "MyInstance", L_big, l_big, d_big)

solve_csp_file("my_instance.txt"; m_max=400)


  Objective = 4.0   Status = OPTIMAL   Time = 0.013 s
  Capacity = 61 | n = 100
  Objective (primal bound): 131.0
  Status: OPTIMAL | Time: 0.13 s
